In [10]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
# MODEL = "llama2"



In [11]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("gpt"):
  model = ChatOpenAI(
    model=MODEL,
    api_key=OPENAI_API_KEY
  )
  embeddings = OpenAIEmbeddings()
else:
  model = Ollama(
    model=MODEL
  )
  embeddings = OllamaEmbeddings()

In [12]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("PUT_YOUR_PDF.pdf")
pages = loader.load_and_split()
pages

[Document(page_content="SeanA.Blair+(65)8511-8496|sablair@gmail.com|LinkedIn\nPROFILEExperiencedSoftwareEngineerwithexpertiseindevelopinginnovativesolutionsandcontributingtothesuccessfuldeploymentofproducts.Skilledincollaboratingwithexternalpartnersandengagingwiththedevelopercommunitytoenhanceandscaleprojects.Possessestechnicalproficiencyinvarioustechnologies,includingReactJS,ReactNative,C#,HackLang,.NETCore,Postgres,Node.JS,ASP.NETMVC,WindowsForms,BorlandDelphiandMicrosoftSQLServer.\nKEYSKILLS1.Collaboration2.PartnerEngagement3.TrainingandDevelopment4.ProblemSolving5.CommunicationandReporting\nPROFESSIONALEXPERIENCE\nMeta,SingaporeJun2017-Jun2023PartnerEngineer\nInmyroleasaPartnerEngineer,IworkedwithMeta'sstrategicpartnerstointegrateMeta'sdeveloperproductsintothoseofthepartner.AsIwasgenerallythebridgebetweenourpartnersandourproductteams,itwasimportantthatIwasabletounderstandthechallengesthepartnerwasfacingandhowourproductcouldmeetthisneed.Inthecasewheretherewasagap,ifpossibleIwouldsee

In [13]:
from langchain.prompts import PromptTemplate

template = """
Answer the question as truthfully as possible using the provided context. If you don't know the answer,
just say that you don't know, don't try to make up an answer.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)


In [15]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = prompt | model | parser
chain.input_schema.schema() # helpful for debugging to tell me what the chain needs to work


{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [16]:
chain.invoke({
  "context": "Optimus Prime is a red semi truck and leader of the autobots.",
  "question": "What kind of vehicle is the leader of the autobots?"
})

python(75598) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


'The leader of the autobots is a red semi truck.'

In [17]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [18]:
retriever = vectorstore.as_retriever()
retriever.invoke("Partner Engineer")

[Document(page_content="SeanA.Blair+(65)8511-8496|sablair@gmail.com|LinkedIn\nPROFILEExperiencedSoftwareEngineerwithexpertiseindevelopinginnovativesolutionsandcontributingtothesuccessfuldeploymentofproducts.Skilledincollaboratingwithexternalpartnersandengagingwiththedevelopercommunitytoenhanceandscaleprojects.Possessestechnicalproficiencyinvarioustechnologies,includingReactJS,ReactNative,C#,HackLang,.NETCore,Postgres,Node.JS,ASP.NETMVC,WindowsForms,BorlandDelphiandMicrosoftSQLServer.\nKEYSKILLS1.Collaboration2.PartnerEngagement3.TrainingandDevelopment4.ProblemSolving5.CommunicationandReporting\nPROFESSIONALEXPERIENCE\nMeta,SingaporeJun2017-Jun2023PartnerEngineer\nInmyroleasaPartnerEngineer,IworkedwithMeta'sstrategicpartnerstointegrateMeta'sdeveloperproductsintothoseofthepartner.AsIwasgenerallythebridgebetweenourpartnersandourproductteams,itwasimportantthatIwasabletounderstandthechallengesthepartnerwasfacingandhowourproductcouldmeetthisneed.Inthecasewheretherewasagap,ifpossibleIwouldsee

In [19]:
from operator import itemgetter

chain = (
  {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
  | prompt
  | model
  | parser
)

chain.invoke({"question": "Does he have experience working in Augmented Reality?"})

'Yes, based on the provided context, Sean A. Blair has experience working in Augmented Reality. This can be inferred from his role as a Partner Engineer at Meta where he worked with notable brands to find ways and create prototypes on how Augmented Reality and Stories can give their products more visibility. Additionally, he gave multiple presentations on Meta Spark AR in APAC to hundreds of developers.'

In [20]:
questions = [
  "Does he have experience working in Augmented Reality?",
  "How long has he been working at Meta?",
  "Does he have any experience working with mobile?",
  "What was his compensation while working at Meta?",
]


for question in questions:
  print(f"Question: {question}")
  print(f"Answer: {chain.invoke({'question': question})}")
  print()

Yes, based on the provided context, Sean A. Blair has experience working in Augmented Reality. He worked with notable brands to find ways and create prototypes on how Augmented Reality and Stories can give their products more visibility.
Sean A. Blair has been working at Meta from June 2017 to June 2023, which is a total of 6 years.
Yes, based on the provided context, Sean A. Blair has experience working with mobile technologies such as React Native, Swift, and Android in his role as a Partner Engineer at Meta.
I don't know.


In [21]:
for s in chain.stream({"question": "Summarize his role at Meta"}):
  print(s, end="", flush=True)

Sean A. Blair worked at Meta as a Partner Engineer from June 2017 to June 2023. In this role, he collaborated with strategic partners to integrate Meta's developer products, presented MetaSparkAR to developers in APAC, served as a primary contact for partners using Facebook Stories, worked with brands on Augmented Reality prototypes, conducted analysis on product documentation, developed Meta Horizon Worlds, and created a development environment for developers. Technologies used in this role included Hacklang, ReactJS, React Native, Swift, Android, C#, Unity, and SQL.

In [22]:
chain.batch([{"question": q} for q in questions])

['Yes, based on the provided context, Sean A. Blair has experience working in Augmented Reality. He mentioned working with notable brands to find ways and create prototypes involving Augmented Reality and Stories to give their products more visibility during his time as a Partner Engineer at Meta.',
 'Sean A. Blair has been working at Meta from June 2017 to June 2023, which is a total of 6 years.',
 'Yes, based on the provided context, Sean A. Blair does have experience working with mobile. In his role at Meta, he worked on Mobile Development and gave multiple presentations on MetaSparkAR in the APAC region. Additionally, he has expertise in technologies such as React Native, Swift, and Android, which are commonly used for mobile development.',
 "I don't know the answer as the provided context does not mention Sean A. Blair's compensation while working at Meta."]